In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import datetime
import shutil

import numpy as np
import pandas as pd

from library import network as nw
from library import common as cm

In [ ]:
%run setup.py
%load_ext autoreload
%autoreload 2

sns.set(style='darkgrid')
np.random.seed(RANDOM_SEED)

##### Step 0: Set save directory

In [ ]:
%run Load_Clean_aux.py

In [ ]:
if FEATURE_SET == 'normal_feature':
    ori_fea = ['M0', 'tau0_implvol0']
    sub_res = res_dir + 'Network/Normal_Feature/TuneHypers/'

if FEATURE_SET == 'delta_vega':
    ori_fea = ['delta_bs', '1_over_sqrt_tau', 'vega_n']
    sub_res = res_dir + 'Network/Delta_Vega/TuneHypers/'
    
if VIX:
    ori_fea += ['fake_vix']

use_fea = [x + '_t' for x in ori_fea] + ['cp_int']
os.makedirs(sub_res, exist_ok=True)
shutil.copy('setup.py', sub_res)

## Tune hyperparameters

In [ ]:
hypers = {
    'nodes_per_layer': (30, 30),
    'reg_alpha': 1e-2,
    'lr': 1e-4,
    'epochs': 200,
    'outact': 'linear'
}

##### Step 3: Setup hypers and target
Here, we specify hyperparameters, which will be used for training at the next step.

In [ ]:
lab_tune = 'reg_alpha'
value_set = [1e-3, 1e-4, 1e-5, 1e-6]
num_run = NUM_REPEATS
end_period = NUM_PERIOD_END

##### Step 4: Run for each hyper value

In [ ]:
for value in value_set:
    # For instance, we tune regularization alpha
    hypers[lab_tune] = value
    alpha_dir = sub_res + f'para={value:.0e}/'
    
    for k in range(num_run):
        print('This is value={} in repeat {}'.format(value, k))
        alpha_dir_dict = {
            'ckp': alpha_dir + f'run{k}/ckp/',
            'history': alpha_dir + f'/run{k}/history/',
            'plot': alpha_dir + f'/run{k}/plot/'
        }
        for key, value in alpha_dir_dict.items():
            os.makedirs(value, exist_ok=True)
        
        df_return = nw.rolling_net(
            df.copy(),
            ori_fea=ori_fea,
            use_fea=use_fea,
            end_period=end_period,
            hypers=hypers,
            sub_res_dir=alpha_dir_dict,
            tune=True
        )  
        

## Summarize tuning results
This section can be run independent of the above one, if the directory paths are given properly.

In [ ]:
value_set = [1e-3, 1e-4, 1e-5, 1e-6]
num_run = NUM_REPEATS

In [ ]:
df_summary = pd.DataFrame(
    index=range(num_run), 
    columns=[f'para={v:.0e}' for v in value_set])

In [ ]:
for value in value_set:
    for run in range(num_run):
        for i in range(end_period + 1):
            tmp = []
            df_history = pd.read_csv(f'{sub_res}para={value:.0e}/run{run}/history/history{i}.csv', index_col=0)
            tmp.append(df_history['val_mean_squared_error'].min())
        
        df_summary.loc[run, f'para={value:.0e}'] = sum(tmp) / len(tmp)
df_summary = df_summary.astype('float32')

In [ ]:
df_summary.describe()

In [ ]:
df_summary.describe().to_csv(f'{sub_res}summary_metrics.csv')

In [ ]:
with open(f'{sub_res}tuning-setting.txt', 'w') as f:
    f.write(f'Date and time: {datetime.datetime.now()},\n')
    f.write(f'Hypers: {hypers},\n')
    f.write(f'{lab_tune} is being tuned.\n')
    f.write(f'Value set: {value_set},\n')
    f.write(f'End period: {end_period},\n')
    f.write(f'Number of repeats: {num_run}\n')
    f.write(f'{lab_tune} is being tuned!')